In [16]:
import numpy as np
import torch
from transformers import CLIPImageProcessor, CLIPTokenizerFast, CLIPModel
import cv2
import matplotlib.pyplot as plt

device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
def normalize_similarities(scores):
    #clipped_scores = np.clip(scores, 0, np.inf)
    return (scores - scores.min()) / (scores.max() - scores.min())

#similarity computation
def get_embeddings(vid_path):
    #load video
    vid = cv2.VideoCapture(vid_path)

    #load clip model
    checkpoint = "openai/clip-vit-base-patch32"
    image_processor = CLIPImageProcessor.from_pretrained(checkpoint)
    clip = CLIPModel.from_pretrained(checkpoint)
    clip.to(device)

    embeddings = []
    while True:
        okay, frame = vid.read()
        if not okay:
            break
        
        with torch.no_grad():
            img_input = image_processor(images=frame, return_tensors="pt", padding=True).to(device)
            embeddings.append(clip.get_image_features(**img_input).detach().cpu())
    
    return embeddings

In [18]:
prompts = ["plane", "person", "windsock", "cloud", "fluff", "candy", "mosquito", "book"]
video_features = get_embeddings("/home/marina/Vidéos/dsgg/cut5.mp4")

In [19]:
with open("cut5.npy", 'wb') as fp:
    np.save(fp, np.array(video_features) )

/tmp/ipykernel_8892/2881030178.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.save(fp, np.array(video_features) )


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5400,) + inhomogeneous part.

In [27]:
from PIL import Image
from transformers import CLIPProcessor

checkpoint = "openai/clip-vit-base-patch32"
processor = CLIPImageProcessor.from_pretrained(checkpoint)
tokenizer = CLIPTokenizerFast.from_pretrained(checkpoint)
clip = CLIPModel.from_pretrained(checkpoint)
both_processor = CLIPProcessor.from_pretrained(checkpoint)

input_text = tokenizer(text="dog", return_tensors="pt", padding=True)
dog_features = clip.get_text_features(**input_text)

input_text = both_processor(text="plane", images=None, return_tensors="pt", padding=True)
plane_features = clip.get_text_features(**input_text)

input_text = both_processor(text="airplane", return_tensors="pt", images = None, padding=True)
airplane_features = clip.get_text_features(**input_text)

img = Image.open("airplane.png")
print(type(img))
image_prompt = both_processor(images = img, text=None, return_tensors="pt", padding=True)#['pixel_values']
image_features = clip.get_image_features(**image_prompt)

input_text = both_processor(text="windsock", images=None, return_tensors="pt", padding=True)
windsock_features = clip.get_text_features(**input_text)

text_text = torch.nn.functional.cosine_similarity(airplane_features, plane_features)
plane_img = torch.nn.functional.cosine_similarity(image_features, plane_features)
airplane_img = torch.nn.functional.cosine_similarity(image_features, airplane_features)

print(plane_features)
print(plane_features.shape)
print (plane_img, airplane_img, text_text)
#for i in range(len(video_features)):
 #   dog_sim = torch.nn.functional.cosine_similarity(video_features[i], dog_features)
  #  plane_sim = torch.nn.functional.cosine_similarity(video_features[i], plane_features)
  #  airplane_sim = torch.nn.functional.cosine_similarity(video_features[i], airplane_features)
  #  image_sim = torch.nn.functional.cosine_similarity(video_features[i], image_features)
  #  windsock_sim = torch.nn.functional.cosine_similarity(video_features[i], windsock_features)
  #  print(dog_sim, plane_sim, airplane_sim, image_sim, windsock_sim)

<class 'PIL.PngImagePlugin.PngImageFile'>
tensor([[ 1.6981e-02,  2.6568e-02,  1.8879e-02,  1.6064e-01, -2.0413e-01,
         -7.2366e-02,  6.9733e-02, -1.1292e+00, -2.4950e-01,  3.8986e-01,
         -3.4463e-01,  3.8627e-02,  1.0020e-01, -1.4741e-01,  2.7803e-01,
         -1.0167e-01,  3.8731e-01, -2.2197e-02, -1.2673e-01, -1.8352e-01,
          3.3995e-01,  3.0566e-01,  1.0492e-01,  1.8506e-02, -3.0351e-01,
         -4.8927e-02, -2.7513e-01,  3.4293e-01,  1.9037e-01,  2.8061e-01,
          1.9474e-01, -1.1052e-01, -3.7759e-03,  2.1314e-01, -3.1525e-01,
          1.7890e-01, -2.1739e-02, -1.2211e-01, -2.0284e-01, -4.1985e-02,
          2.8713e-01, -1.3955e-02,  5.2045e-03,  2.0755e-01,  4.5336e-01,
          1.5053e-01,  2.1676e-01,  2.0452e-01, -5.7958e-02, -1.1162e-02,
          2.4295e-02, -2.8860e-01,  1.9227e-02, -7.2961e-02,  2.0393e-02,
         -5.1770e-02, -4.7329e-02,  1.2304e-01, -4.0251e-01,  2.8067e-01,
          2.5017e-01, -7.8040e-02,  1.1016e-01, -2.9134e-01,  1.1552e-

In [29]:
from numpy.linalg import norm

cos_sim = np.dot(image_features.detach().numpy(), plane_features.detach().numpy().T) / (
    norm(image_features.detach().numpy(), axis=1) * norm(plane_features.detach().numpy(), axis=1)
)

print(cos_sim)

[[0.2705398]]
